# Extended HPS domain
I found that nudging a gaussian bump close to the boundary of the domain can significantly degrade the quality of the HPS's solution, both in the near- and far-field cases. So, I think it would be a good idea to allow for a larger computational domain for HPS to run on compared with the uniform grid from the input.